# Метрические алгоритмы. Практика

В этом домашнем задании вы будете решать задачу классификации бутылок вина по различным характеристикам.

## Импорт библиотек, установка константных значений

In [93]:
import pandas as pd
import numpy as np

In [94]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [95]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [96]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

## Задание 1

Посмотрите на количество классов и количество объектов каждого класса в датасете.

**Вопрос**:  
Сколько классов в задаче?

In [97]:
print(f'{sum(y.unique())} класса')

3 класса


## Задание 2

Мы имеем дело с многоклассовой классификацией. Кроме того, классы не очень хорошо сбалансированы, поэтому для оценки качества модели метрика *accuracy* не подойдет.

Разбейте данные на тренировочную и тестовую части:  
тестовая часть - 25% от всех данных, зафиксируйте `random_state = RANDOM_STATE`.

In [98]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)

**Вопрос:**

Все ли признаки в данных одного масштаба?  
Проверьте это, выведя основные числовые характеристики матрицы `X_train` методом `describe` из библиотеки `pandas`.

По полученной таблице числовых характеристик определите, какой признак измеряется в сотнях?  
(если вариантов несколько, выберите признак с наибольшим средним значением).

In [99]:
X_train.describe().std()

alcohol                          43.273575
malic_acid                       46.195962
ash                              46.306601
alcalinity_of_ash                41.615224
magnesium                        43.410023
total_phenols                    46.300752
flavanoids                       46.304349
nonflavanoid_phenols             46.905216
proanthocyanins                  46.479410
color_intensity                  45.397057
hue                              46.708786
od280/od315_of_diluted_wines     46.203790
proline                         456.995886
dtype: float64

Ответ: Proline

## Задание 3

KNN требует того, чтобы все признаки были одного масштаба, поэтому масштабируйте данные при помощи `StandardScaler`.

Напоминаем, что обучать метод нужно только по тренировочным данным, а применять и к трейну, и к тесту.

После применения `StandardScaler` преобразуйте `X_train` и `X_test` к типу `pd.DataFrame`, названия новых объектов оставьте `X_train` и `X_test`.

In [100]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns) 
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns) 

Обучите KNN с параметрами по умолчанию на тренировочных данных и сделайте предсказание на тесте.

In [101]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
ypred = knn.predict(X_test)

Будем измерять качество модели по метрике weighted $f1$-score.

Чтобы выбрать тип усреднения (micro, macro, weighted) в функции `f1_score` необходимо задать этот тип в гиперпараметре `average`.

Вычислите $f1$-score на тестовых данных.

**Вопрос:**

Чему равен $f1$-score на тестовых данных?

In [102]:
from sklearn.metrics import f1_score

print(f'micro : {f1_score(y_test, ypred, average="micro").round(3)}')
print(f'macro : {f1_score(y_test, ypred, average="macro").round(3)}')
print(f'weighted : {f1_score(y_test, ypred, average="weighted").round(3)}')

micro : 0.956
macro : 0.956
weighted : 0.955


## Задание 4

Попробуем улучшить модель.

Подберите оптимальное количество соседей (`n_neighbors`) из диапазона *от 3 до 30 с шагом 2* и веса соседей (`weights`):  
`uniform`, `distance` по кросс-валидации с тремя фолдами на тренировочных данных.

Используйте `GridSearchCV` и метрику `f1_weighted`.

In [103]:
from sklearn.model_selection import GridSearchCV

param = {'n_neighbors' : np.arange(3, 30, 2),
         'weights' : ['uniform', 'distance']
         }

gs = GridSearchCV(KNeighborsClassifier(), param_grid=param, scoring='f1_weighted')
gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_estimator_)

0.9624902376605162
KNeighborsClassifier(n_neighbors=19)


Возьмите best_estimator_, полученный при обучении GridSearchCV и с помощью него  
сделайте предсказание на тесте и вычислите метрику `f1_weighted`.

In [104]:
ypred = gs.best_estimator_.predict(X_test)
print(f'micro : {f1_score(y_test, ypred, average="micro").round(3)}')
print(f'macro : {f1_score(y_test, ypred, average="macro").round(3)}')
print(f'weighted : {f1_score(y_test, ypred, average="weighted").round(3)}')

micro : 0.956
macro : 0.956
weighted : 0.955


**Вопрос:**

Удалось ли при помощи подбора гиперпараметров улучшить качество модели на тестовых данных?

Ответ: Нет

## Задание 5

Выведите на экран матрицу ошибок.

Используйте модель с подобранными при помощи `GridSearch` гиперпараметрами.


**Вопрос:**  
По этой матрице определите, какие классы между собой путает модель?

In [105]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, ypred)

array([[15,  0,  0],
       [ 1, 16,  1],
       [ 0,  0, 12]], dtype=int64)

## Бонус (эксперименты с LSH)

Скопируйте все функции из [ноутбука в уроке "Быстрый поиск соседей"](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH).

In [106]:
def knn_search(query, data, k=5):

    dists = np.sqrt(np.sum((data - query) ** 2, axis=1))  # вычисляем расстояния от объекта query до всех точек датасета
    inds = np.argsort(dists)  # сортируем по возрастанию расстояний
    inds_k = inds[:k]         # берем top-k точек с наименьшими расстояниями

    return data[inds_k], dists[inds_k], inds_k

def generate_hyperplanes(data, bucket_size=16):
    m = data.shape[0]            # число объектов
    n = data.shape[1]            # число признаков
    b = m // bucket_size         # количество корзин
    h = int(np.log2(b))          # количество гиперплоскостей
    H = rng.normal(size=(h, n))  # гиперплоскости, заданные своими нормалями
    return H

def hamming_hash(data, hyperplanes):
    b = len(hyperplanes)
    hash_key = (data @ hyperplanes.T) >= 0

    dec_vals = np.array([2 ** i for i in range(b)], dtype=int)
    hash_key = hash_key @ dec_vals

    return hash_key

def locality_sensitive_hash(data, hyperplanes):
    hash_vals = hamming_hash(data, hyperplanes)
    hash_table = {}
    for i, v in enumerate(hash_vals):
        if v not in hash_table:
            hash_table[v] = set()
        hash_table[v].add(i)

    return hash_table

def approx_knn_search(query, data, k=5, bucket_size=16):
    candidates = set()

    hyperplanes = generate_hyperplanes(data)
    hash_table = locality_sensitive_hash(data, hyperplanes) # формируем хеш-таблицу по датасету

    query_hash = hamming_hash(query, hyperplanes)
    if query_hash in hash_table:
        candidates = candidates.union(hash_table[query_hash])
    candidates = np.stack([data[i] for i in candidates], axis=0) # находим кандидатов = объекты, попадающие с query в одну корзину

    return knn_search(query, candidates, k=k) # с помощью KNN ищем ближайших соседей только из найденных кандидатов

При помощи `knn_search` найдите ближайших соседей к вину `X_test.iloc[0]` в **тренировочных** данных.

Обратите внимание, что функция `knn_search` принимает на вход `np.array`, а не `pd.DataFrame`. Поэтому переведите аргументы в `np.array`, приписав к необходимому объекту $X$: `X.values`.

In [107]:
%%time
neighbors, dists, inds_k = knn_search(X_test.iloc[0].values, X_train.values)
for dist, inds_k in zip(dists, inds_k):
    print(f"index {inds_k}: dist = {dist}")

index 46: dist = 1.7873320988662762
index 101: dist = 1.9809624052592747
index 108: dist = 2.1601016426727613
index 6: dist = 2.270974564253457
index 127: dist = 2.3117955237515453
CPU times: total: 0 ns
Wall time: 1.05 ms


Выведите на экран признаки объекта `X_test.iloc[0]` и признаки ближайшего найденного соседа.

In [108]:
X_test.iloc[0]

alcohol                         0.806832
malic_acid                      0.651425
ash                             0.707637
alcalinity_of_ash              -1.225688
magnesium                       1.019911
total_phenols                   0.657478
flavanoids                      1.007035
nonflavanoid_phenols           -1.530581
proanthocyanins                 0.089343
color_intensity                 0.037556
hue                             0.002381
od280/od315_of_diluted_wines    1.031337
proline                         0.333672
Name: 0, dtype: float64

In [109]:
X_train.iloc[46]-X_test.iloc[0]

alcohol                         0.701443
malic_acid                      0.812959
ash                            -0.178859
alcalinity_of_ash              -0.578996
magnesium                       0.803048
total_phenols                   0.466371
flavanoids                      0.009983
nonflavanoid_phenols            0.237258
proanthocyanins                 0.730700
color_intensity                 0.000000
hue                            -0.299579
od280/od315_of_diluted_wines    0.233770
proline                        -0.278040
dtype: float64

**Вопрос:**

Можно ли сказать, что в тренировочных данных есть вино, почти такое же как `X_test.iloc[0]`? (все признаки почти одинаковые)

Какое расстояние между объектом запроса и первым ближайшим соседом?

Теперь найдите ближайшего соседа при помощи `approx_knn_search`.

In [110]:
%%time

neighbors, dists, inds_k = approx_knn_search(X_test.iloc[0].values, X_train.values, k=5, bucket_size=16)
for dist, inds_k in zip(dists, inds_k):
    print(f"index {inds_k}: dist = {dist}")

index 4: dist = 1.787332098866276
index 13: dist = 2.3117955237515453
index 5: dist = 2.4714256477971346
index 14: dist = 2.482865265933602
index 2: dist = 2.554033097597557
CPU times: total: 0 ns
Wall time: 2 ms


Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.  
Запустите последнюю ячейку несколько раз и убедитесь, что ближайший сосед находится верно за несколько запусков.